In [1]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [2]:
import os
os.chdir("/project/work/Passenger_Demand")

## 패키지 설치
!pip install seaborn
!pip install haversine
!pip install pyarrow
!pip install multiprocessing_on_dill
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import math
import numbers

from haversine import haversine
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'Nanum Gothic'
sns.set(font="NanumGothic")

In [5]:
%%time
bus_demand_401_10m = pd.read_parquet('model_mr/bus_demand_401_10m.parquet', engine='pyarrow')

CPU times: user 1.35 s, sys: 4.79 s, total: 6.14 s
Wall time: 2.7 s


In [6]:
bus_demand_401_10m.head(20)

,mybi_stop_id,base_ymd,transdate,totalcnt,normalcnt,studentcnt,childcnt,dayofweek,month,day,hour,min,stop_id,longitude,latitude,stop_nm,경도,위도,dist,market_lv2_0,market_lv2_1,market_lv2_2,market_lv2_3,market_lv2_4,market_lv2_5,market_lv2_6,market_lv2_7,market_lv2_8,market_lv2_9,market_lv2_10,market_lv2_11,market_lv2_12,market_lv2_13,market_lv2_14,market_lv2_15,market_lv2_16,market_lv2_17,market_lv2_18,market_lv2_19,market_lv2_20,market_lv2_21,market_lv2_22,market_lv2_23,market_lv2_24,market_lv2_25,market_lv2_26,market_lv2_27,market_lv2_28,market_lv2_29,market_lv2_30,market_lv2_31,market_lv2_32,market_lv2_33,market_lv2_34,market_lv2_35,market_lv2_36,market_lv2_37,market_lv2_38,market_lv2_39,market_lv2_40,market_lv2_41,market_lv2_42,market_lv2_43,market_lv2_44,market_lv2_45,market_lv2_46,market_lv2_47,market_lv2_48,market_lv2_49,market_lv2_50,market_lv2_51,market_lv2_52,market_lv2_53,market_lv2_54,market_lv2_55,market_lv2_56,market_lv2_57,market_lv2_58,market_lv2_59,market_lv2_60,market_lv2_61,market_lv2_62,market_lv2_63,market_lv2_64,market_lv2_65,market_lv2_66,market_lv2_67,market_lv2_68,market_lv2_69,market_lv2_70,market_lv2_71,market_lv2_72,market_lv2_73,market_lv2_74,market_lv2_75,market_lv2_76,market_lv2_77,market_lv2_78,market_lv2_79,market_lv2_80,market_lv2_81,market_lv2_82,market_lv2_83,market_lv2_84,market_lv2_85,market_lv2_86,market_lv2_87,market_lv2_88,market_lv2_89,hospital_lv1_0,hospital_lv1_1,hospital_lv1_2,hospital_lv1_3,hospital_lv1_4,hospital_lv1_5,hospital_lv1_6,hospital_lv1_7,hospital_lv1_8,hospital_lv1_9,hospital_lv1_10,hospital_lv1_11,hospital_lv1_12,hospital_lv1_13,hospital_lv0,market_lv0
0,3100057,20200408,2020-04-08 06:30:00,3,3,0,0,수,4,8,6,30,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
1,3100057,20200408,2020-04-08 06:40:00,1,1,0,0,수,4,8,6,40,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
2,3100057,20200408,2020-04-08 09:00:00,1,1,0,0,수,4,8,9,0,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
3,3100057,20200408,2020-04-08 10:10:00,1,1,0,0,수,4,8,10,10,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
4,3100057,20200408,2020-04-08 11:30:00,1,1,0,0,수,4,8,11,30,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
5,3100057,20200408,2020-04-08 14:10:00,1,1,0,0,수,4,8,14,10,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
6,3100057,20200408,2020-04-08 14:30:00,1,1,0,0,수,4,8,14,30,30619,129.381424,35.552756,현대자동차2공장,129.379093,35.556672,484.0,0,0,0,0,0,0,0,0,0,0,4,1,1,0,0,0,2,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
7,3100057,20200408,2020-

import datetime

a=datetime.datetime.now()-datetime.timedelta(days=1)#하루전
a=datetime.datetime.now()-datetime.timedelta(hours=1)#한시간전
a=datetime.datetime.now()-datetime.timedelta(weeks=1)#일주일전

 

2.달,년

import dateutil.relativedelta import relativedelta
a=datetime.datetime.now()-relativedelta(months=1)#한달전
a=datetime.datetime.now()-relativedelta(years=1)#일년전

In [7]:
import datetime
from functools import partial
from tqdm import tqdm

In [8]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수

        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key

        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
    #func = partial(func, **params)
    
    def map_func(data):
        return data.apply(func, axis = 1, group_data= data.copy() ,**params)
        
        
    with Pool(num_cores) as p:

        pd_result = pd.concat(p.map(map_func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [35]:
def period_before_cnt(data, group_data):
    data_copy = data.copy()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    
    subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()
    before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=1))][col_names].copy()
    
    j = 0
    while len(before_1day) == 0:
        j += 1
        if subset_df['transdate'].min() >= data_copy['transdate'] - datetime.timedelta(days=j):
            before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'])][col_names].copy()
            j = -1
            break
        before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    before_1day['check'] = j
    
    k = 0
    while len(before_1week) == 0:
        k += 1
        if subset_df['transdate'].min() >= data_copy['transdate'] - datetime.timedelta(weeks=k):
            before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'])][col_names].copy()
            k = -1
            break
        before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=k))][col_names].copy()
    before_1week['check'] = k
    
    for name in col_names + ['check']:
        data_copy[name + 'D-1'] = before_1day.reset_index(drop=True).loc[0][name]
        data_copy[name + 'W-1'] = before_1week.reset_index(drop=True).loc[0][name]
    return data_copy

In [36]:
bus_demand_401_10m.shape

(401746, 125)

In [37]:
%%time
bb = parallelize_dataframe(df = bus_demand_401_10m[['stop_id', 'transdate', 'totalcnt', 'normalcnt', 'studentcnt', 'childcnt']]
                                          , func = period_before_cnt, group_keys = 'stop_id', num_cores = 10)

CPU times: user 3.34 s, sys: 3.81 s, total: 7.15 s
Wall time: 52min 3s
